COMP 215 - LAB 2 Records (NEO)
----------------
#### Name: Falak Jain
#### Date: 22 January 2024

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [2]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [3]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2024-01-19 <class 'datetime.date'>
Today is: 2024-01-19


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [ ]:
API_KEY = 'k3fhlue4SVW1pSMpBgKry7ZfFeEXogW4OuBXdw11'  # substitute your API key here

today = str(datetime.date.today())  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:5])

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [12]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

2 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [13]:
# # Ex. 1 your code here
# hazards =  []
# for item in neos:
#   if item['is_potentially_hazardous_asteroid'] is True:
#     hazards.append(item)
# print(f'{len(hazards)} potentially hazardous asteroids identified.')

hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] ]
print(f'{len(hazards)} potentially hazardous asteroids identified.')


2 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [26]:
# Ex. 2 your code here

import datetime, json, requests
from pprint import pprint

API_KEY = 'k3fhlue4SVW1pSMpBgKry7ZfFeEXogW4OuBXdw11'  # substitute your API key here

today = str(datetime.date.today())  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:5])

15 Near Earth Objects found for 2024-01-19
[{'absolute_magnitude_h': 21.17,
  'close_approach_data': [{'close_approach_date': '2024-01-19',
                           'close_approach_date_full': '2024-Jan-19 15:12',
                           'epoch_date_close_approach': 1705677120000,
                           'miss_distance': {'astronomical': '0.4869182247',
                                             'kilometers': '72841929.279301389',
                                             'lunar': '189.4111894083',
                                             'miles': '45261876.0330776082'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '86451.8162166821',
                                                 'kilometers_per_second': '24.0143933935',
                                                 'miles_per_hour': '53717.7965819712'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 1137.6928534001,
  

## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [9]:
# Ex. 3 (challenge) your code here